Stippling Sandbox

In [1]:
from PIL import Image, ImageDraw, ImageFilter
from scipy.spatial import Voronoi, voronoi_plot_2d
import matplotlib.pyplot as plt
import numpy as np
from rasterize import Scanline_Rasterize_Polygon, Sort_Vertices, Bounding_Box, Raster_Centroid, Raster_BBox

In [2]:
def center_rectangle(x,y,l,w,c):
    l = l/2
    w = w/2
    draw.rectangle([x - w, y - l, x + w, y + l],fill=c)

def center_ellipse(x,y,r,c):

    draw.ellipse([x - r, y - r, x + r, y + r],fill=c)


def Outline_Poly(polygon,color,width):
    for i, point in enumerate(polygon):

        k = i + 1
        if k == len(polygon):
            k = 0
        point2 = polygon[k]

        draw.line((point[0],point[1],point2[0],point2[1]),fill=color, width=width)

Image parameters

In [3]:
image_resolution = 1080
image_path = 'sphere.jpg'
weight_image = Image.open(image_path)
weight_image = weight_image.filter(ImageFilter.BoxBlur(5))
res = image_resolution
jitter_mag = 5

scale_factor = image_resolution / res
# img = Image.new('RGB', (image_resolution, image_resolution))
img = Image.open(image_path)
img = weight_image

In [4]:
def Weighted_Raster_Centroid(region,image, image_res):
    cx = 0
    cy = 0
    total_weight = 0
    for pixel in region:
        sample_x = pixel[0]
        sample_y = pixel[1]
        if sample_x >=  image_res or sample_y >=  image_res:
            sampled_value = 125
        elif sample_x < 0 or sample_y < 0:
            sampled_value = 125
        else:
            sampled_value = image.getpixel((sample_x,sample_y))

        weight = 1 - np.mean(sampled_value) / 255

        if weight < 0.05:
            weight = 0
        # if weight < 0.001:
        #     weight = 0.001
        # if total_weight < 0.001:
        #     total_weight = 0.001
        total_weight += weight
        
        cx += pixel[0] * weight
        cy += pixel[1] * weight
    
    if total_weight == 0:
        return False
        
    # if total_weight < 0.001:
    #     total_weight = 0.001

    cx /= total_weight
    cy /= total_weight
    
    return [cx, cy]

In [5]:
def Lloyd_Relax(seeds, res, weight_image):
    vor = Voronoi(seeds)

    polygons = []
    open_cells = []
    for index, region in enumerate(vor.regions):
        poly = []
        if len(region) == 0:
            continue
        elif -1 in region:
            open_cells.append(index)
            continue
        for index in region:
            if index != -1:
                poly.append(vor.vertices[index])
            else:
                print("OUTSIDE")
        
        sorted_poly = Sort_Vertices(poly)

        polygons.append(sorted_poly)

    #Keeping open cells for iterations
    open_cell_points = []
    for region in open_cells:
        if region in vor.point_region:
            index = list(vor.point_region).index(region)
            point = vor.points[index]
            open_cell_points.append(point)
            # center_ellipse(point[0], point[1],10,'blue')

    bbox = [[0,0],[image_resolution,image_resolution]]
    centroids = []
    for polygon in polygons:

        bbox = Raster_BBox(polygon)
        rastered_polygon, scale_factor = Scanline_Rasterize_Polygon(polygon, bbox, res, image_resolution)

        # centroid = Raster_Centroid(rastered_polygon) 
        centroid = Weighted_Raster_Centroid(rastered_polygon, weight_image, image_resolution)
        if centroid:
            if centroid[0] < 0:
                centroid[0] = 0
            elif centroid[0] > image_resolution:
                centroid[0] = image_resolution
            if centroid[1] < 0:
                centroid[1] = 0
            elif centroid[1] > image_resolution:
                centroid[1] = image_resolution

            centroids.append(centroid)
    
    new_seeds = open_cell_points + centroids

    # new_seeds = centroids


    for seed in new_seeds:
        x_jitter = (np.random.rand() - 0.5) * jitter_mag
        y_jitter = (np.random.rand() - 0.5) * jitter_mag
        seed = [seed[0] + x_jitter, seed[1] + y_jitter]

    return new_seeds

Creating seeds

In [6]:
# Finding rough grayscale distribution
seed_img = weight_image



low_count = 0
mid_count = 0
high_count = 0

# for x in range(image_resolution):
#     for y in range(image_resolution):
#         val = weight_image.getpixel((x,y))
#         g = np.mean(val)
#         if g < low_thresh:
#             low_count += 1
#         elif g < mid_thresh:
#             mid_thresh += 1
#         else:
#             high_count += 1
# print(low_count, mid_count, high_count)
# print(low_count + mid_count + high_count)
# print()

In [7]:
seeds = []
startseeds = seeds
point_count = 1000


## Uniform sampling
# for n in range(point_count):
#     rand_x = np.random.rand() * image_resolution
#     rand_y = np.random.rand() * image_resolution
#     seeds.append(np.array([rand_x,rand_y]))

## Rejection sampling
low_thresh = 60
mid_thresh = 125
high_thresh = 225

n = 0
while n < point_count:
    rand_x = np.random.rand() * image_resolution
    rand_y = np.random.rand() * image_resolution
    sampled_value = weight_image.getpixel((rand_x,rand_y))
    if np.mean(sampled_value) < 225:
        seeds.append(np.array([rand_x,rand_y]))
        n += 1

# n = 0
# while n < point_count / 3:
#     rand_x = np.random.rand() * image_resolution
#     rand_y = np.random.rand() * image_resolution
#     sampled_value = weight_image.getpixel((rand_x,rand_y))
#     if np.mean(sampled_value) < 75:
#         seeds.append(np.array([rand_x,rand_y]))
#         n += 1
# n = 0
# while n < point_count / 3:
#     rand_x = np.random.rand() * image_resolution
#     rand_y = np.random.rand() * image_resolution
#     sampled_value = weight_image.getpixel((rand_x,rand_y))
#     if np.mean(sampled_value) < 200:
#         seeds.append(np.array([rand_x,rand_y]))
#         n += 1

# buffer = 5
# count = 20
# spacing = (image_resolution - 2 * buffer) / (count - 1)
# for p in np.linspace(buffer,image_resolution-buffer,count):
#     seeds.append([p,buffer])
#     seeds.append([p,image_resolution - buffer])

# for p in np.linspace(buffer + spacing,image_resolution-buffer,count - 1, endpoint=False):
#     seeds.append([buffer,p])
#     seeds.append([image_resolution - buffer,p])
        
## Seed image
seed_img = Image.open(image_path)
draw = ImageDraw.Draw(seed_img)
for seed in startseeds:
    center_ellipse(seed[0], seed[1],2,'red')
seed_img.save("seed_image.jpg")

In [8]:
for i in range(30):
    new_seeds = Lloyd_Relax(seeds,res, weight_image)

    seeds = new_seeds
    print(f"Iteration {i}")

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4


In [9]:
draw = ImageDraw.Draw(img)

vor = Voronoi(new_seeds)
bbox = [[0,0],[image_resolution,image_resolution]]

polygons = []
region_tracker = 0
open_cells = []
for index, region in enumerate(vor.regions):
    poly = []
    if len(region) == 0:
        continue
    elif -1 in region:
        open_cells.append(index)
        continue
    for index in region:
        if index != -1:
            poly.append(vor.vertices[index])
        else:
            print("OUTSIDE")
    
    sorted_poly = Sort_Vertices(poly)

    polygons.append(sorted_poly)

img = Image.new('RGB', (image_resolution, image_resolution))
draw = ImageDraw.Draw(img)
center_rectangle(image_resolution / 2,image_resolution / 2, image_resolution, image_resolution,'white')

for polygon in polygons:
    g = round(np.random.rand() * 200) + 55

    bbox = Raster_BBox(polygon)
    rastered_polygon, scale_factor = Scanline_Rasterize_Polygon(polygon, bbox, res, image_resolution)

    # for point in rastered_polygon:
    #     # center_rectangle(point[0], point[1], scale_factor -2, scale_factor -2, f'rgb({g},{g},{g})')
    #     center_rectangle(point[0], point[1], scale_factor, scale_factor, f'rgb({g},{g},{g})')
    centroid = Raster_Centroid(rastered_polygon) 
    center_ellipse(centroid[0], centroid[1],2,'black')
    # Outline_Poly(polygon,'gray',2)



In [10]:
img.save('stipple_result.png')